TODO
- fix overlapping_tfs function 

In [6]:
import os
import sys

# Dependency imports

from tensor2tensor.bin import t2t_trainer
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import usr_dir

import tensorflow as tf

# for metrics
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

sys.path.append("../tfti")
import tfti
import tfti_infer



from itertools import combinations
import math
import bisect
import sys
from skpp import ProjectionPursuitRegressor
import numpy as np
import random

sys.path.append("../tfti")
from tfti_batched_inference import *

sys.path.append("../shapley")
import shapley

import time

# Testing MSA

First, run the real Shapley values. This is an example taken from the Wiki https://en.wikipedia.org/wiki/Shapley_value, demonstrating the contributions of workers (w) and bosses (o), where the value function is mp if the boss o is in the set, and 0 otherwise, where m = the number of workers in the set S and p is the profit from each worker.


First, we compute the full Shapley values.

## Full Shapley Computation

In [7]:
from importlib import reload
reload(shapley)
p = 1 # contribution of each worker

# 4 workers and 1 owner
players = np.array(['w','w','w','w','o'])
k = (players == 'w').sum()

def v(S):
    if ('o' in S):
        m = (S == 'w').sum()
        return m * p
    else:
        return 0

orderedList = list([i for i in range(len(players))])
R_orderings = shapley.power_set(orderedList)    
characteristic_function = list(map(lambda x: v(players[x]), R_orderings))

# compute the actual Shapley value
shapleys = shapley.compute_shapley_values(players, characteristic_function)
print("Shapley values for all workers are %s" % shapleys)

Shapley values for all workers are [0.5, 0.5, 0.5, 0.5, 2.0]


### Analyzing the reuslts

As you can see, all workers have a contribution of 0.5, and the boss ('o') has a contribution of 2.

In [8]:
print("Had to compute %i instances of the value function " % (len(R_orderings)))
print(shapleys)
assert(shapleys[0] == p/2)
assert(shapleys[4] == (k * p)/2)

Had to compute 32 instances of the value function 
[0.5, 0.5, 0.5, 0.5, 2.0]


## Computing the Predicted Shapley Values

Next, we compute the Shapley values on predicted value functions using the MSA algorithm. The main idea is that we do not need to compute all 31 instances of the value function. Instead, we will only compute 20 instances and use these to predict the missing value functions. This is useful because in our case, the value function is prediction the model and we have many TFs to predict on.

In [9]:
# Create all possible orderings of the players
orderedList = list([i for i in range(len(players))])
R_orderings = list(enumerate(shapley.power_set(orderedList)))

# choose random subset of R_orderings
sample_size = 20
R_sampled_orderings = random.sample(R_orderings, sample_size)
characteristic_function = np.zeros(len(R_orderings))

# compute the characteristic function for sampled orderings
# in our case, this will be computed from evaluating our model, building a mask from the current ordering
train_data = np.zeros((sample_size, len(players)))
y = np.zeros(sample_size)

for i, x in enumerate(R_sampled_orderings):
    y[i] = v(players[x[1]])
    for j in x[1]:
        train_data[i,j] = 1
    characteristic_function[x[0]] = y[i] 
    

# build a matrix that is #random calcs by len(players)  and predict on v
estimator = ProjectionPursuitRegressor()
estimator.fit(train_data, y)

# predict the missing v(s) 
missing_R_orderings = [x for x in R_orderings if x not in R_sampled_orderings]
for x in missing_R_orderings:
    prediction_vector = np.zeros(len(players))
    for j in x[1]:
        prediction_vector[j] = 1
    characteristic_function[x[0]] = estimator.predict(np.matrix(prediction_vector))


# compute the actual Shapley value
shapleys = shapley.compute_shapley_values(players, characteristic_function)

In [10]:
print("Had to compute %i instances of the value function " % (len(R_sampled_orderings)))
print(shapleys)

Had to compute 20 instances of the value function 
[0.4127064531292884, 0.40945359593800823, 0.4128494995656109, 0.36382321344992297, 1.866972059724756]


# LOADING IN DATA

In [11]:
config = get_config(
    problem="genomics_binding_deepsea_gm12878",
    model="tfti_transformer",
    hparams_set="tfti_transformer_base",
    hparams="",
    checkpoint_path="/data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001",

)

preprocess_batch_fn = get_preprocess_batch_fn(config)
inference_fn = get_inference_fn(config)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-05-02 16:24:39,634] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-05-02 16:24:39,636] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-05-02 16:24:40,268] Building model body


INFO:tensorflow:Transforming body output with binary_class_label_modality_512.top


[2018-05-02 16:24:46,196] Transforming body output with binary_class_label_modality_512.top


INFO:tensorflow:Restoring parameters from /data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001


[2018-05-02 16:24:47,358] Restoring parameters from /data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001


# Load in Generator Data for Validation

In [12]:
# batch_size=1

problem_str="genomics_binding_deepsea_gm12878"
model_str="tfti_transformer"
hparams_set_str="tfti_transformer_base"
hparams_str=""
checkpoint_path="/data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001"

# infer_fn = tfti_infer.get_infer_fn(
#     problem="genomics_binding_deepsea_gm12878",
#     model="tfti_transformer",
#     hparams_set="tfti_transformer_base",
#     hparams="",
#     checkpoint_path="/data/akmorrow/tfti/t2t_train/6-64-25/model.ckpt-210001",
#     batch_size=batch_size
# )



In [13]:
tmp_dir = os.path.expanduser("/data/epitome/tmp/")

config = tfti_infer.get_config(problem_str, model_str, hparams_set_str, hparams_str, checkpoint_path)
problem, model, hparams = tfti_infer.get_problem_model_hparams(config)
generator = problem.generator(tmp_dir, is_training=False)
generator_list = list(generator)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-05-02 16:24:47,731] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-05-02 16:24:47,733] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-05-02 16:24:50,284] Generated 5000 examples.


INFO:tensorflow:Generated 6000 examples.


[2018-05-02 16:24:50,762] Generated 6000 examples.


INFO:tensorflow:Generated 7000 examples.


[2018-05-02 16:24:51,258] Generated 7000 examples.


In [27]:

def pseudo_batch(x, n):
    """Yields the value x n-times."""
    for _ in range(n):
        yield x
        


In [110]:
# reload(tfti_infer)
cell_type_1 = "GM12878"
cell_type_2 = "H1-hESC"


marks = get_tfs(problem, cell_type_1, cell_type_2)
marks_str = '\t'.join(marks)
print(marks)

# get all combs up to 2. This should take about 10 hours.
depth = 2
power_set = shapley.power_set(marks, depth=depth)

batch_size = 64
inputs  = np.array(list(map(lambda x: x['inputs'], generator_list)))
targets = np.array(list(map(lambda x: x['targets'], generator_list)))

f= open(f"shapley_values_64_25_gm12878_depth_{depth}.txt","w+")
f.write(f"permutation\t{marks_str}\taverageAuROC\n")

for set_ in power_set[1:2]:

    start = time.time()

    tf.logging.info("Computing average auROC for set %s" % set_)
    # select marks for this run
    selected_marks = [m for m in marks if m in set_]
    
    keep_mask = tfti_infer.get_keep_mask_for_marks(problem, selected_marks, cell_type_1)
    
    # instantiate labels and predictions for this set
    labels_numpy = np.zeros((len(generator_list), len(marks) ))
    predictions_numpy = np.zeros((len(generator_list), len(marks) ))
    
    for i in range(0, len(generator_list), batch_size):
        batch_keep_mask = pseudo_batch(keep_mask, batch_size)
        
        batch = preprocess_batch_fn(
            inputs[i:i+batch_size],
            targets[i:i+batch_size],
            batch_keep_mask

        )
        response = inference_fn(batch)
        labels_numpy[i:i+batch_size] = response['labels'].reshape((batch_size, len(marks)))
        predictions_numpy[i:i+batch_size] = response['predictions'].reshape((batch_size, len(marks)))
            
    end = time.time()
    elapsed = end - start
    print(f"Completed validation set in {elapsed} seconds")
    
    roc_aucs = []
    for i in range(len(marks)):
        # Compute micro-average ROC area for all marks
        fpr, tpr, _ = roc_curve(labels_numpy[:,i], predictions_numpy[:,i])
        roc_auc = auc(fpr, tpr)
        roc_aucs.append(roc_auc)
        
    roc_auc_str = '\t'.join(str(x) for x in roc_aucs)
    
    # filter out nans to compute auc
    roc_aucs = np.array(roc_aucs)
    average_roc = roc_aucs[np.logical_not(np.isnan(roc_aucs))].mean()
    tf.logging.info("Computed average auROC: %s" % (average_roc))
    
    # save values
    f.write("%s\t%s\t%s\n" % (selected_marks, roc_auc_str, average_roc))
    f.flush()
    
f.close()

INFO:tensorflow:Marks for CellType GM12878: [(204, 'GM12878|ATF2|None'), (205, 'GM12878|ATF3|None'), (207, 'GM12878|BCL11A|None'), (410, 'GM12878|BRCA1|None'), (210, 'GM12878|CEBPB|None'), (412, 'GM12878|CHD1|None'), (413, 'GM12878|CHD2|None'), (127, 'GM12878|CTCF|None'), (53, 'GM12878|DNase|None'), (128, 'GM12878|EZH2|None'), (212, 'GM12878|Egr-1|None'), (216, 'GM12878|GABP|None'), (420, 'GM12878|JunD|None'), (421, 'GM12878|Max|None'), (423, 'GM12878|Mxi1|None'), (223, 'GM12878|NRSF|None'), (428, 'GM12878|Nrf1|None'), (229, 'GM12878|Pol2-4H8|None'), (230, 'GM12878|Pol2|None'), (436, 'GM12878|RFX5|None'), (235, 'GM12878|RXRA|None'), (233, 'GM12878|Rad21|None'), (437, 'GM12878|SIN3A|None'), (236, 'GM12878|SIX5|None'), (237, 'GM12878|SP1|None'), (238, 'GM12878|SRF|None'), (240, 'GM12878|TAF1|None'), (442, 'GM12878|TBP|None'), (241, 'GM12878|TCF12|None'), (243, 'GM12878|USF-1|None'), (444, 'GM12878|USF2|None'), (244, 'GM12878|YY1|None'), (447, 'GM12878|Znf143|None'), (725, 'GM12878|c-Myc|

[2018-05-02 17:38:27,425] Marks for CellType GM12878: [(204, 'GM12878|ATF2|None'), (205, 'GM12878|ATF3|None'), (207, 'GM12878|BCL11A|None'), (410, 'GM12878|BRCA1|None'), (210, 'GM12878|CEBPB|None'), (412, 'GM12878|CHD1|None'), (413, 'GM12878|CHD2|None'), (127, 'GM12878|CTCF|None'), (53, 'GM12878|DNase|None'), (128, 'GM12878|EZH2|None'), (212, 'GM12878|Egr-1|None'), (216, 'GM12878|GABP|None'), (420, 'GM12878|JunD|None'), (421, 'GM12878|Max|None'), (423, 'GM12878|Mxi1|None'), (223, 'GM12878|NRSF|None'), (428, 'GM12878|Nrf1|None'), (229, 'GM12878|Pol2-4H8|None'), (230, 'GM12878|Pol2|None'), (436, 'GM12878|RFX5|None'), (235, 'GM12878|RXRA|None'), (233, 'GM12878|Rad21|None'), (437, 'GM12878|SIN3A|None'), (236, 'GM12878|SIX5|None'), (237, 'GM12878|SP1|None'), (238, 'GM12878|SRF|None'), (240, 'GM12878|TAF1|None'), (442, 'GM12878|TBP|None'), (241, 'GM12878|TCF12|None'), (243, 'GM12878|USF-1|None'), (444, 'GM12878|USF2|None'), (244, 'GM12878|YY1|None'), (447, 'GM12878|Znf143|None'), (725, 'GM12

['ATF2', 'ATF3', 'BCL11A', 'BRCA1', 'CEBPB', 'CHD1', 'CHD2', 'CTCF', 'DNase', 'EZH2', 'Egr-1', 'GABP', 'JunD', 'Max', 'Mxi1', 'NRSF', 'Nrf1', 'Pol2-4H8', 'Pol2', 'RFX5', 'RXRA', 'Rad21', 'SIN3A', 'SIX5', 'SP1', 'SRF', 'TAF1', 'TBP', 'TCF12', 'USF-1', 'USF2', 'YY1', 'Znf143', 'c-Myc', 'p300']
INFO:tensorflow:Computing average auROC for set ['ATF2']


[2018-05-02 17:38:28,314] Computing average auROC for set ['ATF2']


Completed validation set in 46.68943524360657 seconds
INFO:tensorflow:Computed average auROC: 0.8113998834615973


/data/akmorrow/anaconda2/envs/python36/lib/python3.6/site-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
[2018-05-02 17:39:15,091] Computed average auROC: 0.8113998834615973


In [40]:
def get_tfs(problem, cell_type_1, cell_type_2):
    items = problem.get_overlapping_indices_for_cell_type(cell_type_1, cell_type_2)[1]
    return list(map(lambda x: x[1].split('|')[1], items))


In [63]:
targets = np.array(list(map(lambda x: x['targets'], generator_list)))

In [100]:
roc_aucs = np.array(roc_aucs)
x= np.array2string(roc_aucs, precision=3, separator='\t')[1:][:-1]

'0.969840356434741\t0.9626767764465282\t0.717659128377894\t0.755714659572096\t0.6608722615979381\t0.33207228614307155\t0.852525631407852\t0.818784392196098\t0.8801328597530242\t0.9615089554108217\t0.9906578947368421\t0.7052520482272548\t0.965710675078621\t0.8049897221819763\t0.9446392785571142\t0.7090840840840841\t0.8289036776605838\tnan\t0.9050471292380475\t0.9793772098332358\t0.5058674108260885\t0.770267322413041\t0.5922763141348089\t0.5258956416960685\t0.9567371891170089\t0.9645382028859296\t0.7734605632169366\tnan\t0.7909421363294514\t0.8953428747236719\tnan\t0.8499699359565959\t0.8566993710691824\t0.9452613502200279\t0.7920889312444795'

0.8113998834615973

'ATF2/tATF3/tBCL11A/tBRCA1/tCEBPB/tCHD1/tCHD2/tCTCF/tDNase/tEZH2/tEgr-1/tGABP/tJunD/tMax/tMxi1/tNRSF/tNrf1/tPol2-4H8/tPol2/tRFX5/tRXRA/tRad21/tSIN3A/tSIX5/tSP1/tSRF/tTAF1/tTBP/tTCF12/tUSF-1/tUSF2/tYY1/tZnf143/tc-Myc/tp300'